<font color="red" >

# Asset Management & Sustainable Finance - Final Examination
</font>

**Importing necessary libraries**

In [2]:
import pandas as pd
import numpy as np

import cvxopt as opt # library for convex optimization

<font color="red">

## 1. Portfolio optimization and risk budgeting
</font>

The vector of the expected returns is : 
$$ \mu = \left( 0.05, 0.05, 0.06, 0.04, 0.07\right)$$

The vector of the standard deviations is :
$$ \sigma = \left( 0.20, 0.22, 0.25, 0.18, 0.45\right)$$

The correlation matrix of the asset returns is :
$$ \mathbb{C} = \left( \rho_{i,j} \right) = \begin{pmatrix} 1 & & & & \\ 0.5 & 1 & & & \\ 0.3 & 0.3 & 1 & & \\ 0.6 & 0.6 & 0.6 & 1 & \\ 0.4 & 0.3 & 0.7 & 0.3 & 1 \end{pmatrix} = \frac{cov(\mu_i, \mu_j)}{\sigma_i \sigma_j}$$

And the covariance matrix of the asset returns is :
$$ \Sigma = \mathbb{E} \left[ \left( R - \mu \right) \left( R - \mu \right)^T \right] = cov\left(\mu_i, \mu_j \right) = \sigma \mathbb{C} \sigma^T $$

In [19]:
mu = np.array([0.05, 0.05, 0.06, 0.04, 0.07]) # expected returns
correlation = np.array([[1, 0.5, 0.3, 0.6, 0.4],
                [0.5, 1, 0.3, 0.6, 0.3],
                [0.3, 0.3, 1, 0.6, 0.7],
                [0.6, 0.6, 0.6, 1, 0.3],
                [0.4, 0.3, 0.7, 0.3, 1]],
                dtype=np.float64,)       # correlation matrix
sigma = np.array([0.2, 0.22, 0.25, 0.18, 0.45]) # standard deviations

r_f = 0.02 # risk-free rate

<font color="green">

#### 1.(a) Covariance matrix
</font>

In [20]:
# calculate the covariance matrix
covariance = np.zeros_like(correlation)
for i in range(len(correlation)):
    for j in range(len(correlation)):
        covariance[i][j] = correlation[i][j] * sigma[i] * sigma[j]
print(covariance)

[[0.04    0.022   0.015   0.0216  0.036  ]
 [0.022   0.0484  0.0165  0.02376 0.0297 ]
 [0.015   0.0165  0.0625  0.027   0.07875]
 [0.0216  0.02376 0.027   0.0324  0.0243 ]
 [0.036   0.0297  0.07875 0.0243  0.2025 ]]


<font color="green">

#### 1.(b) Sharpe ratio
</font>

The Sharpe ratio of an asset is a performance metric of an investment that adjustss the returns of an investment for the risk-free rate of return. 

The Sharpe ratio is defined as :
$$ S_i = \frac{\mu_i - r_f}{\sigma_i}$$

We compute the Sharpe ratio for each asset:

In [23]:
S = (mu - r_f) / sigma
print("The Sharpe ratios vector:")
print(S)

The Sharpe ratios vector:
[0.15       0.13636364 0.16       0.11111111 0.11111111]


<font color="green">

#### 2. Long/Short MVO portfolios
</font>

2.(a) 
The general formulation of a QP problem is:

$$ x^* = \argmin_x \left( \frac{1}{2} x^{\mathsf T} Q x - x^{\mathsf T} R \right)$$

$$\text{u.c.} \quad Sx \leq T $$

Which corresponds to : 

$$ x^* = \argmin_x \left( \frac{1}{2} x^{\mathsf T} Q x - x^{\mathsf T} R \right)$$

$$\text{u.c.} \left \{ \begin{array}{ccc} A x & = & B \\ C x & \leq & D \\ x^- & \leq & x & \leq &  x^+ \end{array} \right.$$

with $$ Sx \leq T \iff \begin{bmatrix} - A \\ A \\ C \\ - I_n \\ I_n \end{bmatrix} x \leq \begin{bmatrix} - B \\ B \\ D \\ - x^- \\ x^+ \end{bmatrix}$$


We consider the following problem  of the mean-variance optimization problem with a long/short constraint on the weights of the assets: 

$$ x^* = \argmin_x \left( \frac{1}{2} x^{\mathsf T} \Sigma x - \gamma x^{\mathsf T} (\mu - r1_5) \right)$$

$$\text{s.t.} \left \{ \begin{array}{ccc} \sum_{i=1}^{n} x_i = & 1 \\ -10 & \leq & x & \leq &  10 \end{array} \right.$$

In [25]:
def solve_qp(Q, R, A=None, b=None, C=None, D=None, x_min=None, x_max=None):
    """
    Solve a quadratic programming problem:
        minimize    (1/2)x^T Q x - x^T R
        subject to  Ax = b
                    Cx <= D
                    x^- <= x <= x^+
    Parameters:
        Q: numpy array (n x n)
            Symmetric positive semi-definite matrix defining the quadratic term.
        R: numpy array (n x 1)
            Linear term in the objective function.
        A: numpy array (p x n), optional
            Matrix defining the equality constraints.
        b: numpy array (p x 1), optional
            Vector defining the equality constraints.
        C: numpy array (m x n), optional
            Matrix defining the inequality constraints.
        D: numpy array (m x 1), optional
            Vector defining the inequality constraints.
        x_min: numpy array (n x 1), optional
            Vector defining the lower bounds for each variable.
        x_max: numpy array (n x 1), optional
            Vector defining the upper bounds for each variable.
    Returns:
        sol: dict
            Dictionary containing the solution:
                - 'x': optimal solution vector
                - 'optimal_value': optimal value of the objective function
    """
    n = Q.shape[0]
    
    # Constructing the quadratic objective term
    P = opt.matrix(0.5 * (Q + Q.T))
    q = opt.matrix(-R)
    
    # Constructing the inequality constraints
    if C is not None and D is not None:
        G = opt.matrix(C)
        h = opt.matrix(D)
    else:
        G = opt.matrix(0.0, (0, n))
        h = opt.matrix(0.0, (0, 1))
    
    # Constructing the equality constraints
    if A is not None and b is not None:
        A_eq = opt.matrix(A)
        b_eq = opt.matrix(b)
    else:
        A_eq = opt.matrix(0.0, (0, n))
        b_eq = opt.matrix(0.0, (0, 1))
    
    # Constructing the bounds
    if x_min is not None and x_max is not None:
        G_bounds = opt.matrix(-1.0 * np.eye(n))
        h_min = opt.matrix(-1.0 * x_min)
        G_bounds = opt.matrix(np.eye(n))
        h_max = opt.matrix(x_max)
        G = opt.matrix([G, G_bounds])
        h = opt.matrix([h, h_max])
    
    sol = opt.solvers.qp(P, q, G, h, A_eq, b_eq)
    
    return {
        'x': np.array(sol['x']),
        'optimal_value': sol['primal objective']
    }

<font color="red">

## 2. Equity portfolio optimization with net zero objectives
</font>

In [35]:
beta = np.array([0.95, 1.05, 0.45, 1.40, 1.15, 0.75, 1.00, 1.20, 1.10, 0.8, 0.7]).reshape(-1, 1)
sigma_mat = np.array([0.262, 0.329, 0.211, 0.338, 0.231, 0.259, 0.265, 0.271, 0.301, 0.274, 0.228])
sigma_mat = D**2 # to the power of 2

sigma_m = 0.2

cov_mat = (sigma_m**2) * beta.dot(beta.T) 
for i in range(len(cov_mat)):
    cov_mat[i][i] += D[i]

In [36]:
corr_mat = np.zeros_like(cov_mat)

for i in range(len(corr_mat)):
    for j in range(i, len(corr_mat)):
        corr_mat[i, j] = cov_mat[i, j] / (sigma_mat[i, i] * sigma_mat[j, j])
        corr_mat[j, i] = corr_mat[i, j]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [37]:
sector_volatility = np.sqrt(np.diag(cov_mat))
sector_volatility

array([0.32364178, 0.39030885, 0.22939268, 0.43891229, 0.32597699,
       0.29930085, 0.33200151, 0.36199586, 0.37282838, 0.31729482,
       0.26755186])

### Question 1.b

The volatility of the benchmark can be computed as follows:
$$ \sigma_{\text{benchmark}} = \sqrt{b^{\mathsf T} \Sigma b}$$
where $b$ is the vector of the benchmark weights.

In [38]:
# 1.(b)
b = np.array([8.20, 12.30, 6.90, 3.10, 13.20, 12.60, 10.20, 23.00, 4.50, 2.80, 3.20])
b = b / 100 # convert from percentage to decimal
sigma_benchmark = np.sqrt((b.T).dot(cov_mat).dot(b))
print("The volatility of the benchmark portfolio is: ", sigma_benchmark)

The volatility of the benchmark portfolio is:  0.22211447894047792


The benchmark volatility arises from its specific asset composition, determined by pre-defined weights across various sectors or assets, and is calculated using the covariance matrix of returns and these weights. It captures the integrated risk of its constituent assets. 

Conversely, market volatility $\sigma_m$ represents a more comprehensive risk measure, often reflected by a market index or broad market proxy, providing a macro-level perspective on overall market risk, as opposed to the specific risk profile of any individual portfolio composition.

## Question 2.a

The investor's objective is to is to minimize the volatility of the tracking error relative to the benchmark and to meet the decarbonization constraint based on Scope 1 and 2 emissions.

The investor’s decarbonization pathway follows the CTB trajectory, meaning that the carbon in- tensity of the investor’s portfolio at time t must be less than a threshold $\mathcal{CI}^⋆ (t)$. 

- **Decarbonization constraint** based on Scope 1 and 2 emissions:
$$ \mathcal{CI}(t,w) \leq \mathcal{CI}^⋆ (t) := (1-30\%)(1-7\%)^t \mathcal{CI}(b)$$

**General definition of the QP Problem**: 

$$ w^* = \argmin \frac{1}{2} w^\mathsf{T} Qw - w^\mathsf{T}R$$

$$\text{s.t.} \left \{ \begin{array}{ccc} A x & = & B \\ C x & \leq & D \\ x^- & \leq & x & \leq &  x^+ \end{array} \right.$$


**where in our case** : 
- the equality constraint is the budget constraint $ \left( \sum_{i=1}^n w_i = 1  \right) $ : $ A = 1_n^\mathsf{T} $ and $ B = 1 $
- the bounds correspond to the no short-selling restriction (long-only constraint) $ \left( 0 \leq w_i \leq 1 \right) $ : $ w^- = 0_n $ and $ w^+ = 1_n $
- minimmization of the tracking error volatility : $ Q = \Sigma $ and $ R = 0_n $
- the decarbonization constraint : $ C = \mathcal{CI}^\mathsf{T} $ and $ D = \mathcal{CI}^⋆ $